In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as manimation
import io





class Ising:

    _steps = int(2e6)

    def __init__(self, temp, initial_state='r', size=(100,100), show=False):
        
        self.T = temp
        self.sqr_size = size
        self.size = size[0]
        self.initialize(initial_state)

    def initialize(self, initial_state):
        #initialize rand system (to equilibrate) or a fully magnetized system (to deteriorate)  
        if initial_state == 'r':
            system = np.random.randint(0, 1+1, self.sqr_size)
            system[system==0] = -1
        else:
            system = np.ones(self.sqr_size)
        self.system = system

    def periodic_bound(self, i):
        #periodic boundary condition heck if a lattice site coordinate falls out of bounds. If it does, 
        apply periodic boundary condition 
        if i+1 > self.size-1:
            return 0
        if i-1 < 0:
            return self.size-1
        else:
            return i

    def _energy(self, x, y):
        #Calc energy of spin interaction at one lattice point (x,y) with four nearest neighbors
        return -2*self.system[x,y]*(
                    self.system[self.periodic_bound(x-1), y] 
                    + self.system[self.periodic_bound(x+1), y] 
                    + self.system[x, self.periodic_bound(y-1)] 
                    + self.system[x, self.periodic_bound(y+1)]
                    )

    @property #backward compatible
    def internal_energy(self):
        e=0; E=0; E_2=0

        for i in range(self.size):
            for j in range(self.size):   
                e = self._energy(i, j)
                E += e
                E_2 += e**2

        U = (1./self.size**2)*E
        U_2 = (1./self.size**2)*E_2 

        return U, U_2

    @property
    def magnetization(self):
        """Find the overall magnetization of the system
        """
        return np.abs(np.sum(self.system)/self.size**2)
    

    def simulate(self, video=True):
        """Run the simulation
        """

        FFMpegWriter = manimation.writers['ffmpeg']
        writer = FFMpegWriter(fps=10)

        plt.ion()
        fig = plt.figure()

        with writer.saving(fig, "ising.mp4", 100):
            for step in range(self._steps):
                #pick random lattice point
                x,y = np.random.randint(0, self.size, 2)

                #calc energy of a flipped spin
                E = -1*self._energy(x, y)

                print(E)
                #conditions for whether or not spin will flip (preferred state)
                if E <= 0.:
                    self.system[x,y]*=-1
                elif np.exp(-E/self.T) > np.random.random():
                    self.system[x,y]*=-1

                if step % (self._steps//75) == 0: 
                    print("Step {:.2E} / {:.2E}".format(step, self._steps), end='\t')
                    print("Percent completed {:.2f}%".format(100*step/self._steps), end='\t')
                    print("Net Magnetization: {:.2f}".format(self.magnetization))
                    if video:
                        img = plt.imshow(self.system, interpolation='nearest')
                        writer.grab_frame()
                        img.remove()
                
            print("...done")

        plt.close('all')


lattice = Ising(temp=.5, initial_state="r", size=(100,100))
lattice.simulate()